# **<h1><center><span style="color:#3775a8"><u>SORBONNE DATA ANALYTIC : Data Cleaning</u></span></center></h1>**
# **<span style="color:#3775a8">Découverte de Pandas</span>**

**Bienvenue dans ce module !**

Pandas est l'un des packages les plus utilisées pour la manipulation de données en Python bien qu'il en existe d'autres (ex: datatable, vaex, ...).

Comme vous l'avez vu dans la vidéo, le data cleaning est la clé pour réussir une analyse et/ou une modélisation prédictive. Voir l'ensemble des fonctions de pandas serait totalement inutile. Le plus important est de comprendre le fonctionnement du package et de savoir ce que l'on souhaite faire pour ensuite aller chercher l'information qui y réfère dans la documentation.
De plus, je vous conseille de ne jamais utiliser une fonction sans regarder en amont les paramètres de la fonction.

Vous allez voir dans cette formation comment utiliser pandas pour **nettoyer vos données** de manière à les rendre modélisables.

Avant toutes choses, voici les packages et les versions que j'utilise: 
- numpy==1.19.1
- pandas==1.1.0

**Attention**, si vous n'avez pas strictement les même, il est possible que vos output ne soient pas exactement les mêmes !

## **<span style="color:#3775a8">0. Importer les packages</span>**

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

path_data = '~/Mettez/Votre/Chemin/Data/'

## **<span style="color:#3775a8">1. Suppression des variables sans dictionnaire de données et/ou que vous ne comprenez pas</span>**

L'objectif sera de nettoyer la base de donnée afin de pouvoir prédire la Region en fonction des différents éléments naturels.

Voici les données que nvus allez utiliser:

In [ ]:
df = pd.read_csv(path_data + 'base.csv')
df.sample(n=10, random_state=893717398)

**Dictionnaire:**
- Date: Date de la mesure
- Vitesse_Vent_Moyen_10min: Vitesse moyenne du vent
- Humidité: Pourcentage d'humidité
- Precipitation_derniere_heure: 
- Nom: Ville
- Temperature_dC: Température en degrés Celsius
- Altitude: Altitude de la ville
- Longitude: Longitude
- Latitude: Latitude
- Region : Region
- Mois: Moi de l'année 1: Janvier à 12: Décembre

Ici, nous n'avons pas d'information sur la variable **Precipitation_derniere_heure** et imaginons que personne ne sache ce à quoi elle réfère, vous pouvez la supprimer.
De plus, l'**Altitude**, la **Longitude** et la **Latitude** sont **hors périmètre** car ce ne sont pas des éléments naturels.
De plus, le **Mois** peut être obtenu directement à partir de la date donc nous allons le supprimer pour cet exercice

In [ ]:
cols_to_drop_without_dictionnary = ['Precipitation_derniere_heure', 'Altitude', 'Longitude', 'Latitude', 'Mois']
df.drop(columns=cols_to_drop_without_dictionnary, axis=1, inplace=True)

## **<span style="color:#3775a8">2. Parser les dates</span>**

In [ ]:
# Ici, seule la variable Date est une date
print("Format de la date avant d'être parser: " + str(df['Date'].dtypes))

df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%dT%H:%M:%S")

print("\nFormat de la date après d'être parser: "+str(df['Date'].dtypes))

## **<span style="color:#3775a8">3. Splitter la base de données 1/2</span>**

Vous allez utiliser les données de 2011 à 2017 pour l'entrainement du modèle et les données de 2018 à 2020.

In [ ]:
train = df[df['Date'] < '2018-01-01 00:00:00']
test = df[df['Date'] >= '2018-01-01 00:00:00']

## **<span style="color:#3775a8">4. Hypothèse d’indépendance des observations et traitement des doublons</span>**

Cette base de données a été construite en input pour garder uniquement 8 villes afin de ne pas être trop lourde en terme de Mb.
Selon la méthodologie du data cleaning, vous devriez avoir une ligne pour par ville afin de pouvoir modéliser. Mais 8 villes n'est pas suffisant.
Donc vous allez considérer pour la suite du notebook que toutes les observations sont indépendantes les unes des autres.

Cependant, un bon exercice pour pratiquer ce point 3 serait d'utiliser la stratégie 4 en créant les variables suivantes:
- Vitesse_Vent_Moyen_10min_Semestre1
- Vitesse_Vent_Moyen_10min_Semestre2
- Humidite_Semestre1
- Humidite_Semestre2
- Temperature_dC_Semestre1
- Temperature_dC_Semestre2
Agrégées par la valeur **médiane**.
Vous aurez alors 8 observations pour chaque Nom et 6 variables.


Pour la suite du notebook, je considère que toutes les obsevations sont indépendantes et je supprime le **Nom**

In [ ]:
train.drop(columns=['Nom'], axis=1, inplace=True)

## **<span style="color:#3775a8">5. Splitter la base de données 2/2</span>**
Cette fois-ci, vous allez splitter aléatoirement votre base de train en 2 avec un ratio de 60-40:
- train: 60%
- validation: 40%

In [ ]:
validation = train.sample(frac=0.4,random_state=893717398)
train=train.drop(validation.index)

## **<span style="color:#3775a8">6. Supprimer les constantes</span>**
Un simple head permet de voir tout de suite qu'aucune variable n'est constante.

Essayez de créer une boucle ou une fonction qui déterminerait le nombre de valeurs uniques pour chaque variable.

In [ ]:
train.sample(n=10, random_state=893717398)

## **<span style="color:#3775a8">7. Gestion des valeurs manquantes</span>**
Voir les valeurs manquantes en nombre et en pourcentage:

In [ ]:
for i in train.columns:
    print('Nombre de valeurs manquantes pour la variable '+str(i)+' : '+str(train[i].isna().sum()))
    
for i in train.columns:
    print('Pourcentage de valeurs manquantes pour la variable '+str(i)+' : '+str(round((train[i].isna().sum())*100/len(train),2))+'%')

Ici, nous l'objectif est de modéliser la région en fonction des variables d'éléments naturels.
Compte tenu que Region est votre target et non une variable explicative comme le sont Date, Vitesse_Vent_Moyen_10min, Humidite et Temperature_dC; et que l'on ne sait pas à quelle région correspond une observation manquante, il est possible de supprimer les observations.

Pour ce qui est des autres cas, compte tenu que le pourcentage est très bas, vous pouvez procéder à une simple imputation par la médiane.

Un bon exercice afin de pratiquer la manipulation de données ainsi que la data visualization que vous verrez au prochain cours serait de trouver une valeur plus probable que la médiane, bien que cela ne changerait pas fondamentalement les résultats.

In [ ]:
train.dropna(subset=['Region'], axis=0, inplace=True)

vent_median=train['Vitesse_Vent_Moyen_10min'].median()
humidite_median=train['Humidite'].median()
temperature_median=train['Temperature_dC'].median()

train['Vitesse_Vent_Moyen_10min'].fillna(vent_median, inplace=True)
train['Humidite'].fillna(humidite_median, inplace=True)
train['Temperature_dC'].fillna(temperature_median, inplace=True)

In [ ]:
for i in train.columns:
    print('Nombre de valeurs manquantes pour la variable '+str(i)+' : '+str(train[i].isna().sum()))
    
for i in train.columns:
    print('Pourcentage de valeurs manquantes pour la variable '+str(i)+' : '+str(round((train[i].isna().sum())*100/len(train),2))+'%')

## **<span style="color:#3775a8">8. Analyse descriptive des variables et outliers</span>**
Vous verrez dans le cours consacré à la Data Visualization comment en faire en python grâce au package matplotlib. En attendant, voici une méthode très simple permettant d'avoir une vue facile sur la présence d'outliers.

In [ ]:
train.describe()

## **<span style="color:#3775a8">9. Valider le type des variables</span>**
Extraire et verifier que le type de variable correspond à ce que vous voulez vous permettra de faire des sanity check notament au moment de l'application de votre modèle ainsi que pour confirmer qu'une transformation fait le rendu souhaité.

In [ ]:
train.dtypes

## **<span style="color:#3775a8">10. Cas des variables corrélées</span>**
Ici, vous avez 3 variables conitnues donc 9 paires possibles

In [ ]:
df[['Vitesse_Vent_Moyen_10min', 'Humidite', 'Temperature_dC']].corr(method='pearson')

Ici, aucune variables ne sont trop corrélées entre elles.

## **<span style="color:#3775a8">11. Encoding des variables catégorielles en numériques</span>**
Ici, vous n'avez pas de variables catégorielles donc cette étape n'est pas utile

## **<span style="color:#3775a8">12. Appliquer les transformations sur Validation & Test</span>**
Il faut récupérer toutes les étapes afin que l'algorithme prédictif qui sera entrainé sur le train soit applicable dans les mêmes conditions sur la validation et le test.

In [ ]:
test.drop(columns=['Nom'], axis=1, inplace=True)
validation.dropna(subset=['Region'], axis=0, inplace=True)
test.dropna(subset=['Region'], axis=0, inplace=True)

validation['Vitesse_Vent_Moyen_10min'].fillna(vent_median, inplace=True)
validation['Humidite'].fillna(humidite_median, inplace=True)
validation['Temperature_dC'].fillna(temperature_median, inplace=True)

test['Vitesse_Vent_Moyen_10min'].fillna(vent_median, inplace=True)
test['Humidite'].fillna(humidite_median, inplace=True)
test['Temperature_dC'].fillna(temperature_median, inplace=True)

In [ ]:
# Sauvegarde des données pour le features engineering:
train.to_csv(path_data + 'train.csv', index=False)
validation.to_csv(path_data + 'validation.csv', index=False)
test.to_csv(path_data + 'test.csv', index=False)

A présent, votre base est prête à être modélisée. Vous avez pu voir au travers de toutes les étapes comment nettoyer une base de donnée avec pandas. Comme vous l'avez également vu, il aurait été possible de faire différentes actions que je vous laisse faire de vous même afin de vous exercer.

Vous allez voir dans le prochain notebook consacré au features engineering que vous auriez pu créer encore davantage de variables que les 3 finales que sont la vitesse du vent, l'humidité et la température en °C.

# **<span style="color:#37a871">Félicitations !</span>**